In [3]:
import sys
sys.path.append('c:/Users/rmmoh/code/rmm/advent-rmm-2023')

In [4]:
import numpy as np
import copy
from util.grid import Gridder

In [53]:
#fh = open("input17-1.dat")
fh = open("test.dat")
#fh = open("test2.dat")
#fh = open("test_degeneracy.dat")
rows = fh.readlines()
g = Gridder()
for row in rows:
    g.addrow(row, strip=True)
g.done_building()
g.integerize()

In [54]:
g.shape

(13, 13)

# Part 1

For every cell, there are a fixed set of 48 cells they can potentially reach, via multiple methods
But you can get a minimum cost for each A-B pair


In [96]:
#For every cell, calculate the options and their cost

# (R,L) (1,2,3) (U,D) --> New Pos, Cost
# (U,D) (1,2,3) (R,L) --> New Pos, Cost

grid_height = g.nrows
grid_width = g.ncols
#grid = g.grid

costs = {}
paths = {}

opposite = { (0,1) : (0,-1),
             (0,-1) : (0,1),
             (-1,0) : (1,0),
             (1,0) : (-1,0),
             (0,0) : (0,0)
           }

# 1-step is easy
def get_ones(start):
    newcosts = []
    for step in [(0,-1),(0,1),(-1,0),(1,0)]:
        new = start + step
        r = new[0]
        c = new[1]
        if (r>=0) and (c>=0) and (r<g.nrows) and (c<g.ncols):
            newcosts.append( ((r,c), g.val(new), step ))
    return newcosts           
        
start_tuple = (0,0)
start = np.array(start_tuple)
newcosts = get_ones(start)

for (end_point, cost, steps) in newcosts:
    costs[(start_tuple, end_point)] = cost
    paths[(start_tuple, end_point)] = [[steps]]


In [97]:
for (center, start_tuple) in costs:
    print(center, start_tuple, costs[(center, start_tuple)], paths[(center, start_tuple)])

print(costs)
print(paths)

(0, 0) (0, 1) 4 [[(0, 1)]]
(0, 0) (1, 0) 3 [[(1, 0)]]
{((0, 0), (0, 1)): 4, ((0, 0), (1, 0)): 3}
{((0, 0), (0, 1)): [[(0, 1)]], ((0, 0), (1, 0)): [[(1, 0)]]}


In [98]:
for level in range(g.nrows*2+3):
    print(f"Working level {level}")
    node_pairs = list(costs.keys())
    #node_pairs = sorted((x,(a,b)) for (x,(a,b)) in node_pairs if a+b>1)
    
    for (origin, start_tuple) in node_pairs:            
        start = np.array(start_tuple)
        tmp = get_ones(start)
        
        best_cost = costs[(origin, start_tuple)]
        allsteps = paths[(origin, start_tuple)] 
        num_paths = len(allsteps)

        #print(f"*** {(origin, start_tuple)} -- cost so far: {cost}, allsteps: {allsteps}")       
        for idx in range(num_paths):
#             print("  Steps", steps, ":", origin, start_tuple, cost)
            for (end_point, addl_cost, last_step) in tmp:
                #print("  Origin, end point, start_tuple, addl, last step:", origin, end_point, start_tuple, addl_cost, last_step)           
                if origin==end_point: # or (origin, end_point) in costs:
                    #print(".......skip!")
                    continue

                steps = allsteps[idx]
                
                # Check last three steps
                if len(steps) >= 3:
                    if steps[-1]==last_step and steps[-2]==last_step and steps[-3]==last_step:
                        continue
               
                #print(f"  Cost calc: {best_cost}+{addl_cost}={best_cost+addl_cost} vs. min={costs.get((origin, end_point), 99999)}")                        
                        
                if (best_cost+addl_cost) < costs.get((origin, end_point), 99999):
                    paths[(origin, end_point)] = [steps + [last_step]]
                    costs[(origin, end_point)] = best_cost + addl_cost
                    print("  NEW MINIMUM! ENDPOINT=", end_point, costs[(origin, end_point)])                    
                
                elif (best_cost + addl_cost) == costs.get((origin, end_point), -10000):
                    newpath = steps + [last_step]
                    if newpath not in paths[(origin, end_point)]:
                        paths[(origin, end_point)].append(newpath)
                    print("  DEGENERACY! ENDPOINT=", end_point, costs[(origin, end_point)], len(paths))                                              
                    
                #print("  Newallsteps:", end_point, paths[(origin, end_point)])

Working level 0
  NEW MINIMUM! ENDPOINT= (0, 2) 5
  NEW MINIMUM! ENDPOINT= (1, 1) 6
  NEW MINIMUM! ENDPOINT= (1, 1) 5
  NEW MINIMUM! ENDPOINT= (2, 0) 6
Working level 1
  DEGENERACY! ENDPOINT= (0, 2) 5 5
  DEGENERACY! ENDPOINT= (1, 1) 5 5
  DEGENERACY! ENDPOINT= (2, 0) 6 5
  NEW MINIMUM! ENDPOINT= (0, 3) 8
  NEW MINIMUM! ENDPOINT= (1, 2) 6
  DEGENERACY! ENDPOINT= (1, 2) 6 7
  NEW MINIMUM! ENDPOINT= (2, 1) 7
  NEW MINIMUM! ENDPOINT= (3, 0) 9
Working level 2
  DEGENERACY! ENDPOINT= (0, 2) 5 9
  DEGENERACY! ENDPOINT= (1, 1) 5 9
  DEGENERACY! ENDPOINT= (2, 0) 6 9
  DEGENERACY! ENDPOINT= (0, 3) 8 9
  DEGENERACY! ENDPOINT= (1, 2) 6 9
  DEGENERACY! ENDPOINT= (1, 2) 6 9
  DEGENERACY! ENDPOINT= (2, 1) 7 9
  DEGENERACY! ENDPOINT= (3, 0) 9 9
  NEW MINIMUM! ENDPOINT= (1, 3) 13
  NEW MINIMUM! ENDPOINT= (1, 3) 11
  NEW MINIMUM! ENDPOINT= (2, 2) 11
  DEGENERACY! ENDPOINT= (1, 3) 11 11
  DEGENERACY! ENDPOINT= (2, 2) 11 11
  NEW MINIMUM! ENDPOINT= (3, 1) 11
Working level 3
  DEGENERACY! ENDPOINT= (0, 2)

In [99]:
costs

{((0, 0), (0, 1)): 4,
 ((0, 0), (1, 0)): 3,
 ((0, 0), (0, 2)): 5,
 ((0, 0), (1, 1)): 5,
 ((0, 0), (2, 0)): 6,
 ((0, 0), (0, 3)): 8,
 ((0, 0), (1, 2)): 6,
 ((0, 0), (2, 1)): 7,
 ((0, 0), (3, 0)): 9,
 ((0, 0), (1, 3)): 11,
 ((0, 0), (2, 2)): 11,
 ((0, 0), (3, 1)): 11,
 ((0, 0), (1, 4)): 15,
 ((0, 0), (2, 3)): 16,
 ((0, 0), (3, 2)): 15,
 ((0, 0), (4, 1)): 16,
 ((0, 0), (1, 5)): 20,
 ((0, 0), (0, 4)): 19,
 ((0, 0), (2, 4)): 17,
 ((0, 0), (3, 3)): 21,
 ((0, 0), (4, 2)): 19,
 ((0, 0), (4, 0)): 20,
 ((0, 0), (0, 5)): 22,
 ((0, 0), (2, 5)): 21,
 ((0, 0), (3, 4)): 22,
 ((0, 0), (4, 3)): 25,
 ((0, 0), (5, 2)): 22,
 ((0, 0), (5, 0)): 21,
 ((0, 0), (0, 6)): 24,
 ((0, 0), (2, 6)): 26,
 ((0, 0), (3, 5)): 29,
 ((0, 0), (4, 4)): 28,
 ((0, 0), (5, 3)): 30,
 ((0, 0), (5, 1)): 25,
 ((0, 0), (6, 2)): 27,
 ((0, 0), (6, 0)): 25,
 ((0, 0), (0, 7)): 27,
 ((0, 0), (1, 6)): 27,
 ((0, 0), (2, 7)): 32,
 ((0, 0), (3, 6)): 31,
 ((0, 0), (4, 5)): 33,
 ((0, 0), (5, 4)): 35,
 ((0, 0), (6, 3)): 34,
 ((0, 0), (6, 1)): 2

In [100]:
for x in paths[(0, 0), (12, 12)]:
    print(x)

[(0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0), (0, -1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0)]
[(0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0), (0, -1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0)]
[(0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (0, 1), (1, 0), (0, 1), (0, 1), (1, 0), (1, 0), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0), (0, -1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0)]
[(0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (1, 0), (1, 0), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0), (0, -1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0)]
[(0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (-1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (0, 1), (0

In [104]:
print(g.grid)

[[2 4 1 3 4 3 2 3 1 1 3 2 3]
 [3 2 1 5 4 5 3 5 3 5 6 2 3]
 [3 2 5 5 2 4 5 6 5 4 2 5 4]
 [3 4 4 6 5 8 5 8 4 5 4 5 2]
 [4 5 4 6 6 5 7 8 6 7 5 3 6]
 [1 4 3 8 5 9 8 7 9 8 4 5 4]
 [4 4 5 7 8 7 6 9 8 7 7 6 6]
 [3 6 3 7 8 7 7 9 7 9 6 5 3]
 [4 6 5 4 9 6 7 9 8 6 8 8 7]
 [4 5 6 4 6 7 9 9 8 6 4 5 3]
 [1 2 2 4 6 8 6 8 6 5 5 6 3]
 [2 5 4 6 5 4 8 8 8 7 7 3 5]
 [4 3 2 2 6 7 4 6 5 5 5 3 3]]


In [ ]:
# From each node, don't just do "get_ones"... do the get_twos and get_threes, but stop early if those step on a previously visited node along the way to the end (the end CAN have been visited, I think)


# Part 2

In [18]:
%%time
for level in range(100):
    print(f"Working level {level}")
    tmp = {}
    newcosts = {}
    newpaths = {}
    for (origin, start_tuple), cost in costs.items():
        start = np.array(start_tuple)
        tmp = get_ones(start)
        
        allsteps = paths[(origin, start_tuple)]
        
        mincost = 999999
        newallsteps = []
#         print(f"*** {(origin, start_tuple)} -- cost so far: {cost}, allsteps: {allsteps}")
        
        for steps in allsteps:
#             print("  Steps", steps, ":", origin, start_tuple, cost)
        
            for (end_point, addl_cost, last_step) in tmp:
                if center==end_point or (center, end_point) in costs:
                    continue
                # Check last three steps
                if len(steps) >= 3:
                    if steps[-1]==last_step and steps[-2]==last_step and steps[-3]==last_step:
                        continue
                
#                 print("  End point, addl, last step:", end_point, addl_cost, last_step)
#                 print(f"  Cost calc: {cost}+{addl_cost}={cost+addl_cost} vs. min={mincost}")
                
                if (cost+addl_cost) < mincost:
                    newallsteps = [steps + [last_step]]
                    mincost = cost + addl_cost
#                     print("  NEW MINIMUM!", mincost)                    
                elif (cost + addl_cost)==mincost:
                    newallsteps.append(steps + [last_step])
#                     print("  DEGENERACY!", mincost)                                              
#                 print("  Newallsteps:", newallsteps)
                    
        newcosts[(center, end_point)] = mincost
        newpaths[(center, end_point)] = copy.deepcopy(newallsteps)

    costs.update(newcosts)
    paths.update(newpaths)
#     print("--COSTS--")
#     for x in newcosts.items():
#         print(x)
#     print("--PATHS--")
#     for x in newpaths.items():
#         print(x)
#     print("====================")
    

Working level 0
Working level 1
Working level 2
Working level 3
Working level 4
Working level 5
Working level 6
Working level 7
Working level 8
Working level 9
Working level 10
Working level 11
Working level 12
Working level 13
Working level 14
Working level 15
Working level 16
Working level 17
Working level 18
Working level 19
Working level 20
Working level 21
Working level 22
Working level 23
Working level 24
Working level 25
Working level 26
Working level 27
Working level 28
Working level 29
Working level 30
Working level 31
Working level 32
Working level 33
Working level 34
Working level 35
Working level 36
Working level 37
Working level 38
Working level 39
Working level 40
Working level 41
Working level 42
Working level 43
Working level 44
Working level 45
Working level 46
Working level 47
Working level 48
Working level 49
Working level 50
Working level 51
Working level 52
Working level 53
Working level 54
Working level 55
Working level 56
Working level 57
Working level 58
Working